In [1]:
import pandas as pd

# Load datasets
songs = pd.read_csv('../data/songs.csv')
streams = pd.read_csv('../data/streams1.csv')
users = pd.read_csv('../data/users.csv')

In [6]:
# Prepare the data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
streams['listen_hour'] = pd.to_datetime(streams['listen_time']).dt.hour
full_data = streams.merge(songs, on='track_id').merge(users, on='user_id')

# KPI 1: Hourly unqiue listeners
hourly_unique_listeners = full_data.groupby(['listen_date', 'listen_hour'])['user_id'].nunique().reset_index(name='unique_listeners')

# KPI 2: Top listened Artist of the Hour
artist_listen_counts = full_data.groupby(['listen_date', 'listen_hour', 'artists']).size().reset_index(name='listen_counts')
top_artist = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist = top_artist.rename(columns={'artists': 'top_artist'})

# KPI 3: Listening Sessions per User per Hour
full_data['session_id'] = full_data['user_id'].astype(str) + '-' + full_data['listen_time'].astype(str)
sessions_per_user = full_data.groupby(['listen_date', 'listen_hour', 'user_id']).nunique('session_id').reset_index()
avg_sessions_per_user = sessions_per_user.groupby(['listen_date', 'listen_hour'])['session_id'].mean().reset_index(name='avg_sessions_per_user')

# KPI 4: Hourly Track Diversity Index
track_diversity = full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count']).reset_index()
track_diversity['diversity_index'] = track_diversity['nunique'] / track_diversity['count']

# KPI 5: Most Engaged User Group by Age per Hour
# Assuming age groups are segmented as 18-25, 26-35, etc.
users['age_group'] = pd.cut(users['user_age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])
user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')
most_engaged_group = user_group_engagement.loc[user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()].rename(columns={'age_group': 'most_engaged_age_group'})

# Combine all KPIs into one DataFrame
final_kpis = hourly_unique_listeners.merge(top_artist, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(avg_sessions_per_user, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(track_diversity[['listen_date', 'listen_hour', 'diversity_index']], on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(most_engaged_group[['listen_date', 'listen_hour', 'most_engaged_age_group']], on=['listen_date', 'listen_hour'])

In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   user_id       50000 non-null  int64   
 1   user_name     50000 non-null  object  
 2   user_age      50000 non-null  int64   
 3   user_country  50000 non-null  object  
 4   created_at    50000 non-null  object  
 5   age_group     50000 non-null  category
dtypes: category(1), int64(2), object(3)
memory usage: 2.0+ MB


In [8]:
final_kpis.head()

,listen_date,listen_hour,unique_listeners,top_artist,listen_counts,avg_sessions_per_user,diversity_index,most_engaged_age_group
0,2024-06-25,0,376,Hank Williams,4,1.289894,0.997938,46-55
1,2024-06-25,1,383,BTS,3,1.234987,0.995772,36-45
2,2024-06-25,2,356,Dani Fernández,3,1.311798,0.995717,56-65
3,2024-06-25,3,369,Apollo 440,3,1.276423,0.993631,46-55
4,2024-06-25,4,379,George Jones,5,1.229551,0.995708,26-35
